In [1]:
print('hello, world')

hello, world


# Introduction

This file will go through the steps required for all five preposed versions of RAGSync. The versions included within this file are:

* V1: Self Query Retriever (Most Data intensive)(Chroma Dependent)(Buggy as the SelfQueryRetriever is known to having issues)
* V2: Ensemble Retriever (FAISS Dependent)
* V3: Ensemble Retriever + Multi Query Contruction with RagFusion ReRanking (Best one; Most Compute intensive)(FAISS Dependent)
* V4: Straight Rag (Most simple)(FAISS Dependent)
* V5: Straight Rag with Single Query Contruction (Middle Ground between compute and complexity)(FAISS Dependent)

Futher, theory and the application of said versions will be explained within their sections as well.

The project's running is split into three parts: 
* Indexing: The process of storing the documents into vectors. 
* Retrieval: The process of getting the relevent documents
* Generation: The process of generating the answers based on the user query. 

Let us move to the static setup of system in order get the project working

## Defaults:

This project gives you options for a lot of choices that you would have to take for a RAG Based LLM Application. These choices have a default option and alternate options and you are free to select methods outside of the provided options. Make your choices wisely

# Setup

## Requirements:

Create an new python env and then install the requriments.txt using pip to setup the env. 

By Default, using python 3.8.19 for testing the project

## Storing of Documents:

Structure of the current working directory:

```
working_directory/
│
├── data/
│   ├── RawData/
│   └── SupportingDoc/
├── FinalVer.ipynb
├── Learning_Documentation.md
├── README.md
└── requirements.txt
```

**Drop off your Main Rag files** that you want to fetch the answers from in the **RawData** folder, and **drop off your supporting documents** that the model uses make sense of the contents from your main rag file in the **SupportingDoc** folder.

**NOTE: ONLY .TXT FILES ARE SUPPORTED**

## Setup of Secret Keys:

Read the ```Readme.md``` file in order create the key for AWS BedRock along with Llama3 family of models, along with GROQ Key if your are changing the default to GROQ.

Create an ```.env``` file within the working directory and the store the AWS Keys under the names of: 
* AWS_ACCESS_KEY_ID
* AWS_SECRET_ACCESS_KEY

And if in case of GROQ, store it under the name of: 
* GROQ_API_KEY

No other keys or services are requried for the project.

## Selection of Embedding Function:

An embedding function is what turns textual data into numbers, then later on into vectors. The choice of embedding function is very important. You can choose any one of the below two for your usecase: 
* BGE-Base-en-v1.5: Larger in size, more computation, better results for longer context. 
* BGE-Small-en-v1.5: Smaller in size, less computation, a bit worse than BGE-Base-en-v1.5 in longer context due to smaller embedding size. 

By default, BGE-Base-en-v1.5 is being used, but feel free to change it up to any embedding function within BGE or even any other function from Hugging Face.

When running this code for the first time, it will download the files from Hugging Face, later on when recalling the embedding function, it will call the cached files.

The below two blocks of code is for downloading and loading of BGE-Base-en-v1.5 and BGE-Small-en-v1.5. Run any one and leave out the other.

### Default option

In [34]:
"""
    DEFAULT EMBEDDING FUNCTION OPTION: BGE-BASE-EN-v1.5
    This block of code is to download and load up the embedding function
"""
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs, show_progress=True
)
print(hf_embeddings)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='BAAI/bge-base-en-v1.5' cache_folder=None model_kwargs={'device': 'cpu'} encode_kwargs={'normalize_embeddings': True} query_instruction='Represent this question for searching relevant passages: ' embed_instruction='' show_progress=True


### Alternate Option

In [139]:
"""
    ALTERNATE EMBEDDING FUNCTION OPTION: BGE-SMALL-EN-v1.5
    This block of code is to download and load up the embedding function
"""
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
model_name = "BAAI/bge-small-en-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs, show_progress=True
)
print(hf_embeddings)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='BAAI/bge-small-en-v1.5' cache_folder=None model_kwargs={'device': 'cpu'} encode_kwargs={'normalize_embeddings': True} query_instruction='Represent this question for searching relevant passages: ' embed_instruction='' show_progress=True


## Creation of Vector Database:

The below is the most time consuming part of the project, where you are required to create the vector database. You have two options for your vector Database: 
* ChromaDB: Slower but works with Self Query Retrieval
* FAISS: Faster but doesn't support Self Query Retrieval

By Default, FAISS is recommended unless you want to try out Self Query Retrieval. 

The below two blocks of code is for the creation of FAISS and CHROMADB respectively. Run anyone and leave out the other.

### Default Option

In [18]:
"""
    DEFAULT VECTOR DATABASE OPTION: FAISS
    This block of code is to create the folder and then create the vector database within the folder. 
"""
#The below is to read the documents from RawData and spliting them into chunks.
import os
import shutil
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_community.vectorstores import FAISS
loader = DirectoryLoader(
    "Data\RawData",
    glob="*.txt",
    loader_cls=TextLoader
)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, is_separator_regex=False, separators="\n")
splits = text_splitter.split_documents(documents) #Splits the files into chunks.
print("Files read and split! (RD)")

#The below is to create the folder for the vector database for RawData
#In the case of the folder already existing, it will delete the create an new folder.
FAISS_PATH = os.path.join(os.getcwd(), "Data\FAISS_data")
if not os.path.exists(FAISS_PATH):
    os.makedirs(FAISS_PATH, exist_ok=True) #Creates an new folder
else:
    # Delete all files and folders in FAISS_PATH
    shutil.rmtree(FAISS_PATH)
    os.makedirs(FAISS_PATH, exist_ok=True)
print(f"Directory created: {FAISS_PATH}  (RD)")

#The below is the create the vector database for RawData. This will take a while, depending on your system's specs.
vectorstore = FAISS.from_documents(documents=splits, embedding=hf_embeddings) #This is to create the database from scratch.
vectorstore.save_local(FAISS_PATH)
print("Process done for RawData")


#The below does the same as above, but for the supporting documents. 
SDloader = DirectoryLoader(
    "Data\SupportingDoc",
    glob="*.txt",
    loader_cls=TextLoader
)
SDdocuments = SDloader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
SDsplits = text_splitter.split_documents(SDdocuments)
print("Files read and split! (SD)")

FAISS_SDPATH = os.path.join(os.getcwd(), "Data\FAISS_SDdata")
if not os.path.exists(FAISS_SDPATH):
    os.makedirs(FAISS_SDPATH, exist_ok=True) #Creates an new folder
else:
    # Delete all files and folders in FAISS_PATH
    shutil.rmtree(FAISS_SDPATH)
    os.makedirs(FAISS_SDPATH, exist_ok=True)
print(f"Directory created: {FAISS_SDPATH}  (SD)")

#The below is the create the vector database for RawData. This will take a while, depending on your system's specs.
SDvectorstore = FAISS.from_documents(documents=SDsplits, embedding=hf_embeddings) #This is to create the database from scratch.
SDvectorstore.save_local(FAISS_SDPATH)
print("Process done for SupportingDoc")

Files read and split! (RD)
Directory created: c:\Users\yaesh\OneDrive\Desktop\College\Projects\RAGSync\Data\FAISS_data  (RD)


Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Process done for RawData
Files read and split! (SD)
Directory created: c:\Users\yaesh\OneDrive\Desktop\College\Projects\RAGSync\Data\FAISS_SDdata  (SD)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Process done for SupportingDoc


### Alternate Option

In [ ]:
"""
    ALTERNATE VECTOR DATABASE OPTION: CHROMA
    This block of code is to create the folder and then create the vector database within the folder. 
    Since ChromaDB is being used for the Self Query Retrieval, we have to create and include the meta data.
"""
#The below is to read the documents from RawData and spliting them into chunks.
import os
import shutil
import re
from collections import Counter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_community.vectorstores import Chroma

#The below is the helper functions
def extract_timestamp_and_tags(log_entry):
    timestamp_pattern = r'^(\w+ \d{2} \d{2}:\d{2}:\d{2})'
    tags_pattern = r'\[(.*?)\]'
    key_state_pattern = r'-> (\w+)'

    # Find the timestamp
    match = re.search(timestamp_pattern, log_entry)
    if match:
        timestamp = match.group(1)
        
        # Find the tags
        tags_match = re.findall(tags_pattern, log_entry)
        
        # Assign special variables
        first_tag = tags_match.pop(0) if len(tags_match) > 0 else ''
        thid = tags_match[0].strip('[ThId: ]') if len(tags_match) > 1 else ''
        third_tag = tags_match[1].strip() if len(tags_match) > 2 else ''
        
        # Check for key state
        key_state_match = re.search(key_state_pattern, log_entry)
        key_state = key_state_match.group(1) if key_state_match else ''
        
        # Join remaining tags with spaces
        other_tags = ' '.join(tags_match[2:]) if len(tags_match) >= 3 else ''
        
        return timestamp, first_tag, third_tag, other_tags, key_state, thid
    
    return None, None, None, None, None, None

def extract_timestamps_and_tags_from_log_string(log_string):
    log_entries = log_string.strip().split('\n')
    
    entries = []
    
    for entry in log_entries:
        timestamp, first_tag, third_tag, other_tags, key_state, thid = extract_timestamp_and_tags(entry)
        if timestamp and first_tag and third_tag:
            entries.append((timestamp, first_tag, third_tag, other_tags, key_state, thid))
    
    return entries

def include_MetaData(splits):
    for i in range(len(splits)):
        entries = extract_timestamps_and_tags_from_log_string(splits[i].page_content)
        
        # Combine all timestamps into one list
        all_timestamps = [t[0] for t in entries]
        start_time_stamp = all_timestamps[0]
        end_time_stamp = all_timestamps[-1]
        
        # Extract unique values efficiently
        logtypes = list(filter(None, Counter(t[1] for t in entries).keys()))
        maintags = list(filter(None, Counter(t[2] for t in entries).keys()))
        other_tags = list(filter(None, Counter(t[3] for t in entries).keys()))
        key_states = list(filter(None, Counter(t[4] for t in entries).keys()))
        thids = list(filter(None, Counter(t[5] for t in entries).keys()))
        
        splits[i].metadata["Start_Time_Stamp"] = start_time_stamp
        splits[i].metadata["End_Time_Stamp"] = end_time_stamp
        splits[i].metadata["Log_Type_Tag"] = " , ".join(i for i in logtypes)
        splits[i].metadata["Main_Tag"] = " , ".join(i for i in maintags)
        splits[i].metadata["Other_Tags"] = " , ".join(i for i in other_tags)
        splits[i].metadata["Key_States"] = " , ".join(i for i in key_states)
        splits[i].metadata['THID'] = " , ".join(i for i in thids)


loader = DirectoryLoader(
    "Data\RawData",
    glob="*.txt",
    loader_cls=TextLoader
)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, is_separator_regex=False, separators="\n")
splits = text_splitter.split_documents(documents) #Splits the files into chunks.
include_MetaData(splits)
print("Files read and split! (RD)")

#The below is to create the folder for the vector database for RawData
#In the case of the folder already existing, it will delete the create an new folder.
CHROMA_PATH = os.path.join(os.getcwd(), "Data\CHROMA_data")
if not os.path.exists(CHROMA_PATH):
    os.makedirs(CHROMA_PATH, exist_ok=True) #Creates an new folder
else:
    # Delete all files and folders in FAISS_PATH
    shutil.rmtree(CHROMA_PATH)
    os.makedirs(CHROMA_PATH, exist_ok=True)
print(f"Directory created: {CHROMA_PATH}  (RD)")

#The below is the create the vector database for RawData. This will take a while, depending on your system's specs.
#The below is to create the database from scratch.
vectorstore_SD = Chroma.from_documents(documents=splits, embedding=hf_embeddings, persist_directory=CHROMA_PATH)
print("Process done for RawData")


#The below does the same as above, but for the supporting documents. 
SDloader = DirectoryLoader(
    "Data\SupportingDoc",
    glob="*.txt",
    loader_cls=TextLoader
)
SDdocuments = SDloader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
SDsplits = text_splitter.split_documents(SDdocuments)
print("Files read and split! (SD)")

CHROMA_SDPATH = os.path.join(os.getcwd(), "Data\CHROMA_SDdata")
if not os.path.exists(CHROMA_SDPATH):
    os.makedirs(CHROMA_SDPATH, exist_ok=True) #Creates an new folder
else:
    # Delete all files and folders in FAISS_PATH
    shutil.rmtree(CHROMA_SDPATH)
    os.makedirs(CHROMA_SDPATH, exist_ok=True)
print(f"Directory created: {CHROMA_SDPATH}  (SD)")

#The below is the create the vector database for RawData. This will take a while, depending on your system's specs.
#The below is to create the database from scratch.
vectorstore_SD = Chroma.from_documents(documents=SDsplits, embedding=hf_embeddings, persist_directory=CHROMA_SDPATH)
print("Process done for SupportingDoc")

### Zipping to location

In case of you using the vector database you already have. Take the zip of the databases and then unzip them inside the Data Folder. 

The final directory structure should be: 

```
working_directory/
│
├── data/
│   ├── RawData/
│   ├── SupportingDoc/
│   ├── FAISS_data/ (VectorDatabase of the main files)
│   └── FAISS_SDdata/ (VectorDatabase of the supporting files)
├── FinalVer.ipynb
├── Learning_Documentation.md
├── README.md
└── requirements.txt
```

If you wish to use ChromaDB, make sure that your database have meta data in them as that is required for Self Query Retriever and then change the names of the main file and supporting file folders for the ChromaDB Database as: ```CHROMA_data``` & ```CHROMA_SDdata```

# Main CodeBase

## LLM Setup (Common For All Versions):

There is two services for you to choose from:
* AWS BedRock 
* ChatQROQ

By Default, AWS BedRock is being used for testing, but ChatQROQ can also be used. 

As of for the LLM, any model that you wish to use is fine. By Default, Llama3 8b and mistral.mixtral-8x7b-instruct-v0:1 is being used.

### Default option

In [ ]:
"""
    DEFAULT SERVICE OPTION: AWS BEDROCK
    This block of code is to setup the LLM within AWS Bedrock
"""
import os
import boto3
from dotenv import load_dotenv
from langchain_aws import ChatBedrock

load_dotenv()

aws_access_key = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')

bedrock=boto3.client(service_name="bedrock-runtime", region_name='ap-south-1')
llm = ChatBedrock(model_id="meta.llama3-8b-instruct-v1:0",client=bedrock, model_kwargs={"temperature": 0.1, "top_p": 0.9})
# To Test the model
# llm.invoke("Hello!")

### Alternate option

In [141]:
"""
    DEFAULT SERVICE OPTION: CHAT QROQ
    This block of code is to setup the LLM within CHAT QROQ
"""
import os
import boto3
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()

GROQ_API_KEY = os.getenv('GROQ_API_KEY')

llm = ChatGroq(model="llama3-8b-8192", temperature=0)
# To Test the model
# llm.invoke("Hello!")

## Vector DataBase and Retriever Loading (Common For All Versions):

There is two services for you to choose from:
* FAISS 
* ChromaDB

By Default, FAISS is being used for testing, but ChromaDB can also be used. 

As of for the embedding function, any model that you wish to use is fine. By Default, BGE-Base-en-v1.5 is being used.

### Embedding function with Default and Alternate Option

In [1]:
"""
    USE THE SAME EMBEDDING FUNCTION YOU USED WHILE CREATING VECTOR DATABASE.
    This block of code is to load up the embedding function. 
    Both Alternate and default is mentioned here. Use any one and then comment out the other. 
"""
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

#The below is the BGE-Base-en-v1.5
model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs, show_progress=True
)
print(hf_embeddings)


# OR


#The below is the BGE-Small-en-v1.5
# model_name = "BAAI/bge-small-en-v1.5"
# model_kwargs = {"device": "cpu"}
# encode_kwargs = {"normalize_embeddings": True}
# hf_embeddings = HuggingFaceBgeEmbeddings(
#     model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs, show_progress=True
# )
# print(hf_embeddings)

c:\ProgramData\miniconda3\envs\rag\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='BAAI/bge-base-en-v1.5' cache_folder=None model_kwargs={'device': 'cpu'} encode_kwargs={'normalize_embeddings': True} query_instruction='Represent this question for searching relevant passages: ' embed_instruction='' show_progress=True


### Default and Alternate Option

In [140]:
'''
    This is to use FAISS to load the vectordb. 
    To use, Chroma, uncomment the Chroma code and comment out the FAISS code
'''
import os
import shutil
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores import Chroma

#Below is FAISS Code
FAISS_PATH = os.path.join(os.getcwd(), "Data\FAISS_data")
vectorstore = FAISS.load_local(FAISS_PATH, hf_embeddings, allow_dangerous_deserialization = True)
retriever = vectorstore.as_retriever()
FAISS_SDPATH = os.path.join(os.getcwd(), "Data\FAISS_SDdata")
vectorstore_SD = FAISS.load_local(FAISS_SDPATH, hf_embeddings, allow_dangerous_deserialization = True)
retriever_SD = vectorstore.as_retriever()


# OR


#Below is Chroma Code
# CHROMA_PATH = os.path.join(os.getcwd(), "Data", "CHROMA_data")
# vectorstore = Chroma(persist_directory=CHROMA_PATH, embedding_function=hf_embeddings)
# retriever = vectorstore.as_retriever()
# CHROMA_SDPATH = os.path.join(os.getcwd(), "Data", "CHROMA_SDdata")
# vectorstore_SD = Chroma(persist_directory=CHROMA_SDPATH, embedding_function=hf_embeddings)
# retriever_SD = vectorstore_SD.as_retriever()


## Loading the Documents (Common For All Versions):

In some versions of the project, the documents are required to be feed into the retrieval process. 

So, run this section for all versions

In [142]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader 
from langchain.document_loaders import DirectoryLoader
loader = DirectoryLoader(
    "Data\RawData",
    glob="*.txt",
    loader_cls=TextLoader
)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200,is_separator_regex=False, separators="\n")
splits = text_splitter.split_documents(documents)

loader = DirectoryLoader(
    "Data\SupportingDoc",
    glob="*.txt",
    loader_cls=TextLoader
)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200,is_separator_regex=False, separators="\n")
SDsplits = text_splitter.split_documents(documents)

## V1 : Self Query Retriever

### Imports and Helper Functions

In [117]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

### Prompting and Retrieval

In [137]:
metadata_field_info = [
    AttributeInfo(
        name="Time_Stamp",
        description="Start time of the log entry",
        type="string",
    ),
    AttributeInfo(
        name="Log_Type_Tag",
        description="Type of log (INFO, WARN, ERROR), seperated by comma",
        type="string",
    ),
    AttributeInfo(
        name="Main_Tag",
        description="Primary module or component responsible for the log, seperated by comma",
        type="string",
    ),
    AttributeInfo(
        name="Other_Tags",
        description="Additional details about the log entry, seperated by comma",
        type="string",
    ),
    AttributeInfo(
        name="Key_States",
        description="Important provisioning states achieved during this log chunk, seperated by comma.",
        type="string",
    ),
    AttributeInfo(
        name="THID",
        description="Unique identifier for the logging file, seperated by comma",
        type="string",
    )
]

document_content_description = """Development log entries from IoT devices, including timestamps, log types, and various tags."""

retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)

#To Test
# Question = "Enter your Question here"
# docs = retriever.invoke(Question)

In [133]:
template = """Imagine you are a developer and give me answer to the question based on the context alone.
Since infomation within the context might be not be common knowledge and be known to you, I have included another supporting document for you to understand the context even better:

And, be very direct with your answers. Include futher infomation if needed 

question : {question}

supporting document: {supporting_document}

context: {context}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retriever,
     "supporting_document": retriever_SD | format_docs,
     "question": RunnablePassthrough()} 
    | prompt
    | llm
    | StrOutputParser()
)

### Generation

In [ ]:
Question = "Latest Log"
final_rag_chain.invoke(Question)

## V2 : Ensemble Retriever

### Imports and Helper Functions

In [36]:
import re
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def log_and_retrieve(question):
    print(f"Query passed to retriever: {question}")
    return question
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
def findQuestion(text):
    return re.findall(r'"(.*?)"', text)[0]

### Prompting and Retrieval

In [33]:
bm25_retriever = BM25Retriever.from_documents(splits)

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever], weights=[0.5, 0.5]
)

In [40]:
template1 = '''
Imagine you are my mentor and i was given the task of analyzing IoT Log data and i have many questions that i have to find answer for.
I come to you with one of those many questions and you are going to help me figure out how to answer the questions using the documentation that i am giving you. 

question : {user_query}

documentation: {log_documentation}
'''

prompt1 = ChatPromptTemplate.from_template(template1)

final_rag_chain_QA = (
    {"user_query": RunnablePassthrough(),
     "log_documentation": retriever_SD | format_docs} 
    | prompt1
    | llm
    | StrOutputParser()
)

#To Test
# Question = "Enter your Question here"
# print(final_rag_chain_QA.invoke(Question))

In [ ]:
template2 = '''
My mentor had given me a way to answer my assignment question but i am not sure on how exactly to make sense of it. Could you read my mentor's advice and then boil it down to a single question with all of the technical terms so that i do key word search and find the answer easily?

The advice: '{text}'

note: please provide the finalized question with all the key technical terms enclosed with quotes
'''

prompt2 = ChatPromptTemplate.from_template(template2)

final_rag_chain_QA2 = (
    {"text": final_rag_chain_QA}
    | prompt2
    | llm
    | StrOutputParser()
    | findQuestion
)

#To Test
# Question = "Enter your Question here"
print(final_rag_chain_QA2.invoke(Question))

In [43]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
# RAG
template = """Imagine you are a developer and give me answer to the question based on the context alone.
Since infomation within the context might be not be common knowledge and be known to you, I have included another supporting document for you to understand the context even better:

And, be very direct with your answers. Include futher infomation if needed 

question : {question}

supporting document: {supporting_document}

context: {context}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": ensemble_retriever | format_docs,
     "supporting_document": retriever_SD | format_docs,
     "question": final_rag_chain_QA2 | log_and_retrieve} 
    | prompt
    | llm
    | StrOutputParser()
)

### Generation

In [ ]:
Question = "Enter your Question here"
print(final_rag_chain.invoke(Question))

## V3 : Ensemble Retriever + Multi Query Contruction with RagFusion ReRanking

### Imports and Helper Functions

In [143]:
from langchain.load import dumps, loads
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_Tdocs(docs, k = 10):
    return "\n\n".join(doc[0].page_content for doc in docs[0:k+1])

def createQuestionsList(text):
    return text.split("\n")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def reciprocal_rank_fusion(results, k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents 
        and an optional parameter k used in the RRF formula """
    
    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

### Prompting and Retrieval

In [144]:
bm25_retriever = BM25Retriever.from_documents(splits)

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever], weights=[0.5, 0.5]
)

In [145]:
template = """You are an AI language model assistant. Your task is to generate five
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines. Original question: {question}

Since there are some infomation that should be added to the question in order to provide the best results to retrieve the documents from the vector database. 
You are supposed to include technical keywords to further help the document retrieval process.
As the LLM which will analysis these questions and the retrieved documents have no prior knowledge on the documents nor the question, make the question technical
The required documents are below: 

Required Documents: {context}"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

# getMoreQuestions = ({"context": retriever_SD | format_docs, "question": RunnablePassthrough()} 
#                     | prompt_rag_fusion 
#                     | ChatBedrock(model_id="mistral.mixtral-8x7b-instruct-v0:1",client=bedrock, model_kwargs={"temperature": 0.1, "top_p": 0.9})
#                     | StrOutputParser()  
#                     | createQuestionsList)

# OR 

getMoreQuestions = ({"context": retriever_SD | format_docs, "question": RunnablePassthrough()} 
                    | prompt_rag_fusion 
                    | ChatGroq(model="mixtral-8x7b-32768")  
                    | StrOutputParser()  
                    | createQuestionsList)

#To Test
# Question = "Enter your Question here"
# getMoreQuestions.invoke(Question)

In [146]:
retrieval_chain_rag_fusion = getMoreQuestions | ensemble_retriever.map() | reciprocal_rank_fusion

#To Test
# Question = "Enter your Question here"
# docs = retrieval_chain_rag_fusion.invoke(Question)
# len(docs)

In [147]:
template = """Imagine you are a developer and give me answer to the question based on the context alone.
Since infomation within the context might be not be common knowledge and be known to you, I have included another supporting document for you to understand the context even better:

And, be very direct with your answers. Include futher infomation if needed 

question : {question}

supporting document for you to make better sense of the question : {supporting_document}

context: {context}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion | format_Tdocs, 
     "supporting_document": retriever_SD | format_docs,
     "question": RunnablePassthrough()} 
    | prompt
    | llm
    | StrOutputParser()
)

### Generation

In [ ]:
Question = "Enter your Question here"
print(final_rag_chain.invoke(Question))

## V4 : Straight Rag

### Imports and Helper Functions

In [45]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

### Prompting and Retrieval

In [ ]:
template = '''
Imagine you are a developer and give me answer to the question based on the context alone.
Since infomation within the context might be not be common knowledge and be known to you, I have included another supporting document for you to understand the context even better:

And, be very direct with your answers. Include futher infomation if needed 

question : {question}

supporting document: {supporting_document}

context: {context}
'''

prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "supporting_document": retriever_SD | format_docs , "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

### Generation

In [ ]:
Question = "Enter your Question here"
rag_chain.invoke(Question)

## V5 : Straight Rag with Single Query Contruction

### Imports and Helper Functions

In [116]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

### Prompting and Retrieval

In [ ]:
template = '''
Imagine you are a developer and give me answer to the question based on the context alone.
And, be very direct with your answers. 

And make sure that you do not hallucinate and answer based on the context given below only. If you are unable to answer any part of the answer, say "I don't know" to that part of the answer.

question : {question}

context: {context}
'''
promptAnswer = PromptTemplate.from_template(template)

template2 = '''

You are requested to provide the only answer and nothing else! Make it consise. The question is as follows: 

Imagine you are a developer and you want to prompt a LLM to answer a question for you. But the question has a lot of domain specific infomation which the LLM might not understand. 
So you creating a better question, based on your original question and the documentation that you have wrote during your coding process which explains these domain specific infomation.
You have to give me an question which has the complete context of the documentation along the core question so that your LLM can find everything effectively.

Note: Within the question, you can't say note the documentation as the llm doesn't have access to it, You should pull the relevent info from the documentation to provide me the answers. 

Give few short examples as well within the questions and make question lenghty but not out of context.

question : {question}

documentation: {documentation}
'''
promptQuestion = PromptTemplate.from_template(template2)

Qrag_chain = (
    {"documentation": retriever_SD | format_docs , "question": RunnablePassthrough()}
    | promptQuestion
    | llm
    | StrOutputParser()
)
Arag_chain = (
    {"context": retriever | format_docs,  "question": RunnablePassthrough()}
    | promptAnswer
    | llm
    | StrOutputParser()
)

### Generation

In [ ]:
def RunModel(query):
    Question = Qrag_chain.invoke(query)
    return Arag_chain.invoke(Question)

Question = "Enter your Question here"
print(RunModel(Question))

# Further Testing

## Single Line Chunking for Self Query Retiever.

In [114]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs, show_progress=True
)
print(hf_embeddings)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='BAAI/bge-small-en' cache_folder=None model_kwargs={'device': 'cpu'} encode_kwargs={'normalize_embeddings': True} query_instruction='Represent this question for searching relevant passages: ' embed_instruction='' show_progress=True


In [115]:
"""
    Meant for single line chunking and retriever, using bge-small-en. Not v1.5, just normal
"""
#The below is to read the documents from RawData and spliting them into chunks.
import os
import shutil
import re
from collections import Counter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_community.vectorstores import Chroma
import langchain_core

#The below is the helper functions
def extract_timestamp_and_tags(log_entry):
    timestamp_pattern = r'^(\w+ \d{2} \d{2}:\d{2}:\d{2})'
    tags_pattern = r'\[(.*?)\]'
    key_state_pattern = r'-> (\w+)'

    # Find the timestamp
    match = re.search(timestamp_pattern, log_entry)
    if match:
        timestamp = match.group(1)
        
        # Find the tags
        tags_match = re.findall(tags_pattern, log_entry)
        
        # Assign special variables
        first_tag = tags_match.pop(0) if len(tags_match) > 0 else ''
        thid = tags_match[0].strip('[ThId: ]') if len(tags_match) > 1 else ''
        third_tag = tags_match[1].strip() if len(tags_match) > 2 else ''
        
        # Check for key state
        key_state_match = re.search(key_state_pattern, log_entry)
        key_state = key_state_match.group(1) if key_state_match else ''
        
        # Join remaining tags with spaces
        other_tags = ' '.join(tags_match[2:]) if len(tags_match) >= 3 else ''
        
        return timestamp, first_tag, third_tag, other_tags, key_state, thid
    
    return None, None, None, None, None, None

def extract_timestamps_and_tags_from_log_string(log_string):
    log_entries = log_string.strip().split('\n')
    
    entries = []
    
    for entry in log_entries:
        timestamp, first_tag, third_tag, other_tags, key_state, thid = extract_timestamp_and_tags(entry)
        if timestamp and first_tag and third_tag:
            entries.append((timestamp, first_tag, third_tag, other_tags, key_state, thid))
    
    return entries

def include_MetaData(splits):
    for i in range(len(splits)):
        entries = extract_timestamps_and_tags_from_log_string(splits[i].page_content)
        time_stamp, logtypes, maintags, other_tags, key_states, thids = "None" ,"None" ,"None" ,"None" ,"None", "None" 
        
        # Combine all timestamps into one list
        if (len(entries) > 0 and len(entries[0]) >= 6): 
            time_stamp = entries[0][0]
            logtypes = entries[0][1]
            maintags = entries[0][2]
            other_tags = entries[0][3]
            key_states = entries[0][4]
            thids = entries[0][5]
        
        splits[i].metadata["Time_Stamp"] = time_stamp
        splits[i].metadata["Log_Type_Tag"] = logtypes
        splits[i].metadata["Main_Tag"] = maintags
        splits[i].metadata["Other_Tags"] = other_tags
        splits[i].metadata["Key_States"] = key_states
        splits[i].metadata['THID'] = thids

loader = DirectoryLoader(
    "Data\RawData",
    glob="*.txt",
    loader_cls=TextLoader
)
documents = loader.load()
lines = re.split(r'\r?\n', documents[0].page_content)
splits_lines = []
for i in lines: 
    splits_lines.append(langchain_core.documents.base.Document(page_content = i))
include_MetaData(splits_lines)
print("Files read and split! (RD)")

# The below is to create the folder for the vector database for RawData
# In the case of the folder already existing, it will delete the create an new folder.
CHROMA_PATH = os.path.join(os.getcwd(), "Data\TCHROMA_data")
if not os.path.exists(CHROMA_PATH):
    os.makedirs(CHROMA_PATH, exist_ok=True) #Creates an new folder
else:
    # Delete all files and folders in FAISS_PATH
    shutil.rmtree(CHROMA_PATH)
    os.makedirs(CHROMA_PATH, exist_ok=True)
print(f"Directory created: {CHROMA_PATH}  (RD)")

#The below is the create the vector database for RawData. This will take a while, depending on your system's specs.
#The below is to create the database from scratch.
vectorstore = Chroma.from_documents(documents=splits_lines, embedding=hf_embeddings, persist_directory=CHROMA_PATH)
print("Process done for RawData")


#The below does the same as above, but for the supporting documents. 
SDloader = DirectoryLoader(
    "Data\SupportingDoc",
    glob="*.txt",
    loader_cls=TextLoader
)
SDdocuments = SDloader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
SDsplits = text_splitter.split_documents(SDdocuments)
print("Files read and split! (SD)")

CHROMA_SDPATH = os.path.join(os.getcwd(), "Data\TCHROMA_SDdata")
if not os.path.exists(CHROMA_SDPATH):
    os.makedirs(CHROMA_SDPATH, exist_ok=True) #Creates an new folder
else:
    # Delete all files and folders in FAISS_PATH
    shutil.rmtree(CHROMA_SDPATH)
    os.makedirs(CHROMA_SDPATH, exist_ok=True)
print(f"Directory created: {CHROMA_SDPATH}  (SD)")

# #The below is the create the vector database for RawData. This will take a while, depending on your system's specs.
# #The below is to create the database from scratch.
vectorstore_SD = Chroma.from_documents(documents=SDsplits, embedding=hf_embeddings, persist_directory=CHROMA_SDPATH)
print("Process done for SupportingDoc")

Files read and split! (RD)
Directory created: c:\Users\yaesh\OneDrive\Desktop\College\Projects\RAGSync\Data\TCHROMA_data  (RD)


Batches:   0%|          | 0/47 [00:00<?, ?it/s]

Process done for RawData
Files read and split! (SD)
Directory created: c:\Users\yaesh\OneDrive\Desktop\College\Projects\RAGSync\Data\TCHROMA_SDdata  (SD)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Process done for SupportingDoc
